In [1]:
import os
import re
import glob
import pickle
import numpy as np
import pandas as pd
import pandas.io.common
import warnings
warnings.filterwarnings('ignore')

In [47]:
#### File Path and Variables
loc_path = r'/Users/ritik.pareek/Desktop/' 
path = loc_path + r'/Causal Model/01. Raw Data/05. PRB Data/'
pickle_path = loc_path + 'Causal Model/02. Processed Data/05. PRB Data/'
### Input for the Circle 
Circle= 'bihar'

monthname = "Feb"
year = "2023"

In [4]:
#### Collating Files 
# currently we have colated file but in future run we have to colate the files


allFiles = glob.glob(path + "/*.parquet")
lst=[]
df_master=pd.DataFrame()
for i in allFiles:
    df=pd.read_parquet(i,engine='pyarrow')
    #print(df.shape)
    df_master=df_master.append(df)
    #print(df_master.shape)

In [5]:
df_master.shape

(964536, 11)

In [6]:
df_master.columns

Index(['city', 'component', 'hour', 'prb_20', 'prb_70_90', 'prb_90',
       'ph_mean_prb_dl', 'totprbdlmin_perc', 'totprbdlmax_perc',
       'totprbdltot_perc', 'totprbdlcnt_cnt'],
      dtype='object')

# Removing rows where hour is null

In [7]:
df_master['hour'].unique()

array([19.,  6.,  5., 20., 11., 23., 21., 16., 18., 14.,  7., 13., 12.,
        2.,  3.,  4.,  8., 15., 17.,  0.,  9., 22., 10.,  1., nan])

In [11]:
# Null count
df_master[df_master['hour'].isnull()].shape[0]

336

In [12]:
# dropping the null values
df_master.dropna(subset=['hour'], how='all', inplace=True)

In [25]:
# index reseting
df_master.reset_index(inplace=True, drop=True)

In [13]:
df_master['hour'].unique()

array([19.,  6.,  5., 20., 11., 23., 21., 16., 18., 14.,  7., 13., 12.,
        2.,  3.,  4.,  8., 15., 17.,  0.,  9., 22., 10.,  1.])

In [14]:
df_master.shape

(964200, 11)

# Defining the data types

In [15]:
df_master['city'] = df_master['city'].astype('object')
df_master['component'] = df_master['component'].astype('int64')
df_master['hour'] = df_master['hour'].astype('int64')
df_master['prb_20'] = df_master['prb_20'].astype('int64')
df_master['prb_70_90'] = df_master['prb_70_90'].astype('int64')
df_master['prb_90'] = df_master['prb_90'].astype('int64')
df_master['ph_mean_prb_dl'] = df_master['ph_mean_prb_dl'].astype('float64')
df_master['totprbdlmin_perc'] = df_master['totprbdlmin_perc'].astype('float64')
df_master['totprbdlmax_perc'] = df_master['totprbdlmax_perc'].astype('float64')
df_master['totprbdltot_perc'] = df_master['totprbdltot_perc'].astype('float64')
df_master['totprbdlcnt_cnt'] = df_master['totprbdlcnt_cnt'].astype('float64')

In [16]:
df_master.dtypes

city                 object
component             int64
hour                  int64
prb_20                int64
prb_70_90             int64
prb_90                int64
ph_mean_prb_dl      float64
totprbdlmin_perc    float64
totprbdlmax_perc    float64
totprbdltot_perc    float64
totprbdlcnt_cnt     float64
dtype: object

# Rounding off the data

In [17]:
df_master=df_master.round(2)

In [18]:
df_master.head()

,city,component,hour,prb_20,prb_70_90,prb_90,ph_mean_prb_dl,totprbdlmin_perc,totprbdlmax_perc,totprbdltot_perc,totprbdlcnt_cnt
0,darbhanga,3,19,1,14,15,78.07,29.75,97.83,12541.18,175.80
1,gaya,15,6,15,1,0,20.54,3.67,63.21,2516.73,175.75
2,patna,195,5,36,0,0,9.55,2.60,38.91,1236.47,174.19
3,lakhisarai,24,20,0,5,3,77.74,30.35,96.65,12445.37,175.82
4,gopalganj,20,11,0,6,4,66.61,19.70,97.13,10485.31,175.79


# Setting the column seq

In [19]:
print(df_master.shape)
print(df_master.columns)

(964200, 11)
Index(['city', 'component', 'hour', 'prb_20', 'prb_70_90', 'prb_90',
       'ph_mean_prb_dl', 'totprbdlmin_perc', 'totprbdlmax_perc',
       'totprbdltot_perc', 'totprbdlcnt_cnt'],
      dtype='object')


In [20]:
df_master=df_master[['city', 'component', 'hour', 'prb_20', 'prb_70_90', 'prb_90',
       'ph_mean_prb_dl', 'totprbdlmin_perc', 'totprbdlmax_perc',
       'totprbdltot_perc', 'totprbdlcnt_cnt']]

# Removing right Spaces from the columns

In [21]:
print('Removing right Spaces from the columns')
df_master.columns = df_master.columns.str.rstrip()
df_master.columns = df_master.columns.str.lstrip()
df_master.columns = df_master.columns.str.capitalize()

Removing right Spaces from the columns


# Remove white space and Drop duplicates

In [22]:
## Pre Processing
def remove_whitespaces(df):
    df_obj = df.select_dtypes(['object'])
    df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    df.columns = df.columns.str.strip()
    return df

def drop_duplicate_records(df):
    df.drop_duplicates()
    return df

In [23]:
df_master=remove_whitespaces(df_master)
df_master=drop_duplicate_records(df_master)

In [28]:
df_master.head()

,City,Component,Hour,Prb_20,Prb_70_90,Prb_90,Ph_mean_prb_dl,Totprbdlmin_perc,Totprbdlmax_perc,Totprbdltot_perc,Totprbdlcnt_cnt
0,darbhanga,3,19,1,14,15,78.07,29.75,97.83,12541.18,175.80
1,gaya,15,6,15,1,0,20.54,3.67,63.21,2516.73,175.75
2,patna,195,5,36,0,0,9.55,2.60,38.91,1236.47,174.19
3,lakhisarai,24,20,0,5,3,77.74,30.35,96.65,12445.37,175.82
4,gopalganj,20,11,0,6,4,66.61,19.70,97.13,10485.31,175.79


In [38]:
# no of unique cities
df_master['City'].unique()

array(['darbhanga', 'gaya', 'patna', 'lakhisarai', 'gopalganj', 'chapra',
       'dhanbad', 'jamshedpur', 'bokaro_steel_city', 'giridih', 'barauni',
       'motihari', 'ranchi', 'katihar', 'buxar', 'arrah', 'madhubani',
       'mokameh', 'kishanganj', 'sitamarhi', 'jhumri_tilaiya',
       'samastipur', 'bihar_sharif', 'hazaribag', 'saharsa',
       'muzaffarpur', 'begusarai', 'dumka', 'bettiah', 'bhagalpur',
       'sahibganj', 'hajipur', 'ramgarh', 'supaul', 'jamui', 'saunda',
       'dehri', 'sasaram', 'aurangabad_bi', 'deoghar', 'daltonganj',
       'jamalpur', 'khagaria', 'phusro', 'siwan', 'chaibasa', 'bagaha',
       'jehanabad', 'araria', 'rajgir', 'munger', 'chirkunda', 'nawada',
       'chakradharpur'], dtype=object)

In [52]:
# no of comp in each city
df_master[['City','Component']].groupby('City').nunique()

,Component
City,
araria,80
arrah,123
aurangabad_bi,70
bagaha,47
barauni,105
begusarai,142
bettiah,110
bhagalpur,234
bihar_sharif,130


In [31]:
## Null count
df_master.isnull().sum()

City                0
Component           0
Hour                0
Prb_20              0
Prb_70_90           0
Prb_90              0
Ph_mean_prb_dl      0
Totprbdlmin_perc    0
Totprbdlmax_perc    0
Totprbdltot_perc    0
Totprbdlcnt_cnt     0
dtype: int64

In [37]:
df_master[['Ph_mean_prb_dl','Totprbdlmin_perc','Totprbdlmax_perc','Totprbdltot_perc','Totprbdlcnt_cnt']].describe()

,Ph_mean_prb_dl,Totprbdlmin_perc,Totprbdlmax_perc,Totprbdltot_perc,Totprbdlcnt_cnt
count,964200.000000,964200.000000,964200.000000,964200.000000,964200.000000
mean,42.431351,10.707335,77.284738,6193.779468,168.440480
std,22.069760,9.099064,21.751449,3625.430014,19.131511
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.440000,3.580000,66.680000,2927.585000,175.750000
50%,43.780000,8.380000,86.480000,6244.575000,175.750000
75%,58.780000,14.510000,93.590000,8782.720000,175.790000
max,98.560000,93.280000,99.080000,17246.240000,185.520000


In [48]:
### Saving in Pre Processing Folder
df_master.to_pickle(pickle_path+'{}{}_PRB_data.csv'.format(monthname,year))
print('Data succesfully stored in:',pickle_path+'{}{}_PRB_data.csv'.format(monthname,year))

Data succesfully stored in: /Users/ritik.pareek/Desktop/Causal Model/02. Processed Data/05. PRB Data/Feb2023_PRB_data.csv
